In [1]:
import psycopg2

from itertools import chain
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests 
from datetime import datetime, timedelta
import re
import pytz

In [3]:
cur=conn.cursor()

In [4]:
cur.execute("""SELECT DISTINCT(name) FROM public.cards WHERE rarity IN('mythic','rare') AND setcode IN(SELECT code FROM public.sets WHERE type IN ('expansion','core') and releasedate>=(SELECT releasedate FROM public.sets WHERE mcmname='Return to Ravnica'))""")
cardnames=cur.fetchall()

utc_now = pytz.utc.localize(datetime.utcnow())
pst_now = utc_now.astimezone(pytz.timezone("America/Los_Angeles"))
yesterday=pst_now - timedelta(1)

In [5]:
#check for othercardnames in the name of the set, also remove when ready,release,never is present perhaps?


for outercard in cardnames: #changed from cardnames to cardlist
    #cur.execute("""SELECT id,title,cardset FROM public.transactions WHERE cardname=%s AND possiblybad is FALSE AND othercards IS NOT NULL""",(outercard[0],))
    cur.execute("""SELECT id,title,cardset FROM public.transactions WHERE cardname=%s AND possiblybad IS FALSE AND carddate=%s""",(outercard[0],(yesterday).strftime('%Y-%m-%d')))
    #cur.execute("""SELECT id,title FROM public.transactions WHERE cardname=%s AND possiblybad IS FALSE AND othercards IS NULL""",(outercard[0],))
    #cur.execute("""SELECT id,title,cardset FROM public.transactions WHERE cardname=%s AND possiblybad IS FALSE and othercards IS NOT NULL AND cardset='HOU'""",(outercard[0],))
    transactions=cur.fetchall()
#     cur.execute("""SELECT DISTINCT(printings) FROM public.cards WHERE name=%s""",(outercard[0],))
#     cardprintings=cur.fetchall()
#     cardstring=cardprintings[0][0]
#     setlist=cardstring.split(",")
#     innercards=set()
#     for setname in setlist:
#             cur.execute("""SELECT DISTINCT(name) FROM public.cards WHERE rarity IN('mythic','rare') AND printings ILIKE %s AND setcode IN(SELECT code FROM public.sets WHERE type IN ('expansion','core') and releasedate>=(SELECT releasedate FROM public.sets WHERE mcmname='Return to Ravnica'))""",('%'+setname+'%',))
#             tempcards=set(cur.fetchall())
#             innercards=innercards.union(tempcards)
    for transid,title,cardset in transactions:
        othercards=''
        cur.execute('''SELECT name FROM public.sets WHERE code=%s''',(cardset,))
        fullsetname=cur.fetchall()
        for innername in cardnames:
            if innername[0]==outercard[0]:
                pass
            elif innername[0]=='Hour of Devastation' and cardset=='HOU':
                pass
            elif innername[0].lower() in title.lower() and innername[0].lower() not in outercard[0].lower():
                if len(fullsetname)==0:   
                    if othercards=='':
                        othercards=innername[0]
                    else:
                        othercards+=";"+innername[0]
                elif len(fullsetname)>0 and innername[0].lower() not in fullsetname[0][0].lower():
                    if othercards=='':
                        othercards=innername[0]
                    else:
                        othercards+=";"+innername[0]
        if othercards!='':
            cur.execute("""UPDATE public.transactions SET othercards=%s WHERE id=%s""",(othercards,transid))
        elif othercards=='':
            cur.execute("""UPDATE public.transactions SET othercards=%s WHERE id=%s""",(None,transid))
    conn.commit()
        

In [6]:
cur.close()
conn.close()